# Project 3 Raw to clean

## First, Imports and Reading Data

In [1]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
from scipy import stats
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings('ignore')

import pickle

In [2]:
df = pd.read_csv('NPDB2401.csv')
df

,SEQNO,RECTYPE,REPTYPE,ORIGYEAR,WORKSTAT,WORKCTRY,HOMESTAT,HOMECTRY,LICNSTAT,LICNFELD,...,ACCRRPTS,NPMALRPT,NPLICRPT,NPCLPRPT,NPPSMRPT,NPDEARPT,NPEXCRPT,NPGARPT,NPCTMRPT,FUNDPYMT
0,1,A,301,1991,OK,NaN,NaN,NaN,OK,10,...,0,0,2,0,0,0,0,0,0,NaN
1,2,A,301,1991,OK,NaN,NaN,NaN,OK,10,...,0,0,7,0,0,0,1,0,0,NaN
2,4,A,301,1991,MA,NaN,NaN,NaN,MA,15,...,0,1,1,0,0,0,2,0,0,NaN
3,6,A,301,1990,OK,NaN,NaN,NaN,OK,10,...,0,0,2,0,0,0,0,0,0,NaN
4,8,A,301,1990,OK,NaN,NaN,NaN,OK,10,...,0,0,9,0,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751051,2173175,P,102,2024,NJ,NaN,NaN,NaN,NJ,110,...,0,1,0,0,0,0,0,0,0,0.0
1751052,2173176,C,302,2024,NaN,NaN,CO,NaN,CO,430,...,0,0,1,0,0,0,0,0,0,NaN
1751053,2173177,P,102,2024,CT,NaN,NaN,NaN,NJ,10,...,0,1,0,0,0,0,0,0,0,0.0
1751054,2173178,P,102,2024,MD,NaN,NaN,NaN,NJ,10,...,0,1,0,0,0,0,0,0,0,0.0


### Get only Modern Day Malpractice Payment 

In [3]:
df = df[df['RECTYPE'].isin(['P'])]
df

,SEQNO,RECTYPE,REPTYPE,ORIGYEAR,WORKSTAT,WORKCTRY,HOMESTAT,HOMECTRY,LICNSTAT,LICNFELD,...,ACCRRPTS,NPMALRPT,NPLICRPT,NPCLPRPT,NPPSMRPT,NPDEARPT,NPEXCRPT,NPGARPT,NPCTMRPT,FUNDPYMT
402535,514456,P,101,2004,AZ,NaN,NaN,NaN,AZ,10,...,0,2,2,0,0,0,0,0,0,0.0
402536,514457,P,101,2004,PA,NaN,NaN,NaN,PA,10,...,0,6,0,0,0,0,0,0,0,0.0
402538,514460,P,101,2004,SD,NaN,SD,NaN,SD,651,...,0,1,0,0,0,0,0,0,0,0.0
402551,514475,P,101,2004,NJ,NaN,NaN,NaN,NJ,430,...,0,1,0,0,0,0,0,0,0,0.0
402553,514477,P,101,2004,NH,NaN,NaN,NaN,NH,30,...,0,1,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750946,2173067,P,102,2024,CA,NaN,NaN,NaN,CA,642,...,0,1,0,0,0,0,0,0,0,0.0
1751051,2173175,P,102,2024,NJ,NaN,NaN,NaN,NJ,110,...,0,1,0,0,0,0,0,0,0,0.0
1751053,2173177,P,102,2024,CT,NaN,NaN,NaN,NJ,10,...,0,1,0,0,0,0,0,0,0,0.0
1751054,2173178,P,102,2024,MD,NaN,NaN,NaN,NJ,10,...,0,1,0,0,0,0,0,0,0,0.0


### Get the amount of years of the malpractice record has gone on 

In [4]:
df['MALTIME'] = df['MALYEAR1'] - df['MALYEAR2'].map(lambda x: 0 if pd.isnull(x) else x)
df['MALTIME'] = df['MALTIME'].map(lambda x: int(-1*x) if x < 0 else 0)
df['MALTIME']

402535     0
402536     0
402538     0
402551     0
402553     0
          ..
1750946    0
1751051    0
1751053    0
1751054    0
1751055    0
Name: MALTIME, Length: 258934, dtype: int64

### Let's drop the columns with too many nulls (10% is way too much) 

In [5]:
cols_to_check = df.columns
is_null_col = dict(df[cols_to_check].isnull().apply(lambda x: x.sum() > (509608*.1) ))
null_col = [key for key in is_null_col.keys() if is_null_col[key]]
null_col

['WORKCTRY',
 'HOMESTAT',
 'HOMECTRY',
 'ALEGATN2',
 'MALYEAR2',
 'AAYEAR',
 'AACLASS1',
 'AACLASS2',
 'AACLASS3',
 'AACLASS4',
 'AACLASS5',
 'BASISCD1',
 'BASISCD2',
 'BASISCD3',
 'BASISCD4',
 'BASISCD5',
 'AALENTYP',
 'AALENGTH',
 'AAEFYEAR',
 'AASIGYR']

### SEQNO and RECTYPE are useless. I do not need ids or values that are singular

In [6]:
df.drop(null_col + ['SEQNO', 'RECTYPE', 'MALYEAR1', 'MALYEAR2'], axis=1,inplace=True)
df.dropna(inplace=True)
df

,REPTYPE,ORIGYEAR,WORKSTAT,LICNSTAT,LICNFELD,PRACTAGE,GRAD,ALGNNATR,ALEGATN1,OUTCOME,...,NPMALRPT,NPLICRPT,NPCLPRPT,NPPSMRPT,NPDEARPT,NPEXCRPT,NPGARPT,NPCTMRPT,FUNDPYMT,MALTIME
402535,101,2004,AZ,AZ,10,30.0,1990.0,20.0,306.0,9.0,...,2,2,0,0,0,0,0,0,0.0,0
402536,101,2004,PA,PA,10,50.0,1980.0,1.0,200.0,9.0,...,6,0,0,0,0,0,0,0,0.0,0
402538,101,2004,SD,SD,651,50.0,1970.0,100.0,316.0,9.0,...,1,0,0,0,0,0,0,0,0.0,0
402551,101,2004,NJ,NJ,430,20.0,1990.0,60.0,334.0,3.0,...,1,0,0,0,0,0,0,0,0.0,0
402554,101,2004,TX,TX,10,60.0,1960.0,60.0,108.0,5.0,...,1,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750946,102,2024,CA,CA,642,50.0,2000.0,70.0,500.0,3.0,...,1,0,0,0,0,0,0,0,0.0,0
1751051,102,2024,NJ,NJ,110,50.0,1990.0,10.0,110.0,7.0,...,1,0,0,0,0,0,0,0,0.0,0
1751053,102,2024,CT,NJ,10,30.0,2010.0,70.0,106.0,6.0,...,1,0,0,0,0,0,0,0,0.0,0
1751054,102,2024,MD,NJ,10,20.0,2010.0,70.0,106.0,6.0,...,1,0,0,0,0,0,0,0,0.0,0


## Time to deal with the payments 

### Objectives
   1. Fill in TOTALPMT as PAYMENT
   2. Account for inflation in PAYMENT

### Objective 1 - Fill in TOTALPMT as PAYMENT

In [7]:
df['PAYMENT'] = df['TOTALPMT']
df.drop('TOTALPMT', axis=1,inplace=True)
df

,REPTYPE,ORIGYEAR,WORKSTAT,LICNSTAT,LICNFELD,PRACTAGE,GRAD,ALGNNATR,ALEGATN1,OUTCOME,...,NPMALRPT,NPLICRPT,NPCLPRPT,NPPSMRPT,NPDEARPT,NPEXCRPT,NPGARPT,NPCTMRPT,FUNDPYMT,MALTIME
402535,101,2004,AZ,AZ,10,30.0,1990.0,20.0,306.0,9.0,...,2,2,0,0,0,0,0,0,0.0,0
402536,101,2004,PA,PA,10,50.0,1980.0,1.0,200.0,9.0,...,6,0,0,0,0,0,0,0,0.0,0
402538,101,2004,SD,SD,651,50.0,1970.0,100.0,316.0,9.0,...,1,0,0,0,0,0,0,0,0.0,0
402551,101,2004,NJ,NJ,430,20.0,1990.0,60.0,334.0,3.0,...,1,0,0,0,0,0,0,0,0.0,0
402554,101,2004,TX,TX,10,60.0,1960.0,60.0,108.0,5.0,...,1,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750946,102,2024,CA,CA,642,50.0,2000.0,70.0,500.0,3.0,...,1,0,0,0,0,0,0,0,0.0,0
1751051,102,2024,NJ,NJ,110,50.0,1990.0,10.0,110.0,7.0,...,1,0,0,0,0,0,0,0,0.0,0
1751053,102,2024,CT,NJ,10,30.0,2010.0,70.0,106.0,6.0,...,1,0,0,0,0,0,0,0,0.0,0
1751054,102,2024,MD,NJ,10,20.0,2010.0,70.0,106.0,6.0,...,1,0,0,0,0,0,0,0,0.0,0


### Objective 2 - Account for inflation in PAYMENT

#### What years are in the data frame 

In [8]:
df['ORIGYEAR'].unique()

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int64)

### Inflation for each year and a function that will help with that

In [9]:
def inflation(years, amounts):
    ref_year = {
        2004: 1.66, 2005: 1.61, 2006: 1.56,
        2007: 1.51, 2008: 1.46, 2009: 1.46,
        2010: 1.44, 2011: 1.40, 2012: 1.37,
        2013: 1.35, 2014: 1.33, 2015: 1.33,
        2016: 1.31, 2017: 1.28, 2018: 1.25,
        2019: 1.23, 2020: 1.21, 2021: 1.16,
        2022: 1.07, 2023: 1.03, 2024: 1.00
    }
    return [amounts.iloc[i]*ref_year[years.iloc[i]] for i in range(len(years))]


#### Parse PAYMENT into ints

In [10]:
df['PAYMENT'] = df['PAYMENT'].map(lambda x: int(x[1:]))
df['PAYMENT']

402535       97500
402536      125000
402538      175000
402551       77500
402554      315000
            ...   
1750946      27500
1751051     405000
1751053    1350000
1751054     405000
1751055     945000
Name: PAYMENT, Length: 192098, dtype: int64

In [11]:
df['PAYMENT'] = inflation(df['ORIGYEAR'], df['PAYMENT'])
df['PAYMENT']

402535      161850.0
402536      207500.0
402538      290500.0
402551      128650.0
402554      522900.0
             ...    
1750946      27500.0
1751051     405000.0
1751053    1350000.0
1751054     405000.0
1751055     945000.0
Name: PAYMENT, Length: 192098, dtype: float64

### Next, are there any other useless columns

### Most of the columns categorical, so how many categories are there if it is a categorical column

In [12]:
col_ref = {}
for col in df.columns:
    col_ref[col]  = len(df[col].unique())
col_ref

{'REPTYPE': 2,
 'ORIGYEAR': 21,
 'WORKSTAT': 59,
 'LICNSTAT': 57,
 'LICNFELD': 94,
 'PRACTAGE': 8,
 'GRAD': 12,
 'ALGNNATR': 11,
 'ALEGATN1': 91,
 'OUTCOME': 10,
 'PAYMENT': 2926,
 'PAYNUMBR': 2,
 'NUMBPRSN': 23,
 'PAYTYPE': 5,
 'PYRRLTNS': 8,
 'PTAGE': 12,
 'PTGENDER': 3,
 'PTTYPE': 4,
 'TYPE': 37,
 'PRACTNUM': 139839,
 'ACCRRPTS': 1,
 'NPMALRPT': 66,
 'NPLICRPT': 27,
 'NPCLPRPT': 16,
 'NPPSMRPT': 6,
 'NPDEARPT': 6,
 'NPEXCRPT': 11,
 'NPGARPT': 4,
 'NPCTMRPT': 3,
 'FUNDPYMT': 2,
 'MALTIME': 19}

### Useless Columns 

   - PRACTNUM is an id for practitioner
   - ACCRRPTS has only one unique value
   - ORIGYEAR is the year, which is irrevlent to new data. 

In [13]:
df.drop(['PRACTNUM', 'ACCRRPTS', 'ORIGYEAR'], axis=1,inplace=True)
df

,REPTYPE,WORKSTAT,LICNSTAT,LICNFELD,PRACTAGE,GRAD,ALGNNATR,ALEGATN1,OUTCOME,PAYMENT,...,NPMALRPT,NPLICRPT,NPCLPRPT,NPPSMRPT,NPDEARPT,NPEXCRPT,NPGARPT,NPCTMRPT,FUNDPYMT,MALTIME
402535,101,AZ,AZ,10,30.0,1990.0,20.0,306.0,9.0,161850.0,...,2,2,0,0,0,0,0,0,0.0,0
402536,101,PA,PA,10,50.0,1980.0,1.0,200.0,9.0,207500.0,...,6,0,0,0,0,0,0,0,0.0,0
402538,101,SD,SD,651,50.0,1970.0,100.0,316.0,9.0,290500.0,...,1,0,0,0,0,0,0,0,0.0,0
402551,101,NJ,NJ,430,20.0,1990.0,60.0,334.0,3.0,128650.0,...,1,0,0,0,0,0,0,0,0.0,0
402554,101,TX,TX,10,60.0,1960.0,60.0,108.0,5.0,522900.0,...,1,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750946,102,CA,CA,642,50.0,2000.0,70.0,500.0,3.0,27500.0,...,1,0,0,0,0,0,0,0,0.0,0
1751051,102,NJ,NJ,110,50.0,1990.0,10.0,110.0,7.0,405000.0,...,1,0,0,0,0,0,0,0,0.0,0
1751053,102,CT,NJ,10,30.0,2010.0,70.0,106.0,6.0,1350000.0,...,1,0,0,0,0,0,0,0,0.0,0
1751054,102,MD,NJ,10,20.0,2010.0,70.0,106.0,6.0,405000.0,...,1,0,0,0,0,0,0,0,0.0,0


### Let's rename and add columns

In [14]:
df = df.rename(columns={'REPTYPE' : 'ISINSURE', 'FUNDPYMT' : 'STATEFUND'})

### WKNLICEQ - work (WK) and (N) License (LIC) states are the same (EQ) 

In [15]:
df['ISINSURE'] = df['ISINSURE'].map({101 : 1, 102 : 0})
df.insert(3, 'WKNLICEQ', (df['WORKSTAT'] == df['LICNSTAT']).astype(int))
df

,ISINSURE,WORKSTAT,LICNSTAT,WKNLICEQ,LICNFELD,PRACTAGE,GRAD,ALGNNATR,ALEGATN1,OUTCOME,...,NPMALRPT,NPLICRPT,NPCLPRPT,NPPSMRPT,NPDEARPT,NPEXCRPT,NPGARPT,NPCTMRPT,STATEFUND,MALTIME
402535,1,AZ,AZ,1,10,30.0,1990.0,20.0,306.0,9.0,...,2,2,0,0,0,0,0,0,0.0,0
402536,1,PA,PA,1,10,50.0,1980.0,1.0,200.0,9.0,...,6,0,0,0,0,0,0,0,0.0,0
402538,1,SD,SD,1,651,50.0,1970.0,100.0,316.0,9.0,...,1,0,0,0,0,0,0,0,0.0,0
402551,1,NJ,NJ,1,430,20.0,1990.0,60.0,334.0,3.0,...,1,0,0,0,0,0,0,0,0.0,0
402554,1,TX,TX,1,10,60.0,1960.0,60.0,108.0,5.0,...,1,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750946,0,CA,CA,1,642,50.0,2000.0,70.0,500.0,3.0,...,1,0,0,0,0,0,0,0,0.0,0
1751051,0,NJ,NJ,1,110,50.0,1990.0,10.0,110.0,7.0,...,1,0,0,0,0,0,0,0,0.0,0
1751053,0,CT,NJ,0,10,30.0,2010.0,70.0,106.0,6.0,...,1,0,0,0,0,0,0,0,0.0,0
1751054,0,MD,NJ,0,10,20.0,2010.0,70.0,106.0,6.0,...,1,0,0,0,0,0,0,0,0.0,0


In [16]:
df.to_csv('NPDB2401_Modern_Malpractice_Clean.csv', index=False)